# Optimization notebook
This notebook is to run a multi-objective optimization with the Zambezi model. Two alternatives will be given to that end. First implementation is with the Platypus library which has a fairly simple interface. The second implementation is the EMA Workbench which additionally offers parallelization functionality

In [1]:
import pandas as pd
import numpy as np
import os
os.chdir('../src')
from model_zambezi_OPT import model_zambezi

from platypus import NSGAII, Problem, Real

In [2]:
ZambeziProblem = model_zambezi()

## Platypus (NSGAII)

In [3]:
problem = Problem(ZambeziProblem.Nvar, ZambeziProblem.Nobj)
problem.types[:] = Real(0, 1)
problem.function = ZambeziProblem.evaluate

algorithm = NSGAII(problem=problem, population_size=20)
algorithm.run(100)

In [4]:
objectives_outcome = dict()
for i, column_name in enumerate(['Hydropower','Environment','Irrigation']):
    objectives_outcome[column_name] = [s.objectives[i] for s in algorithm.result]

objectives_df = pd.DataFrame(objectives_outcome)


In [5]:
print(objectives_df)
from various_plots import parallel_plots
parallel_plots(objectives_df)

    Hydropower   Environment  Irrigation
0    15.510580  2.784769e+06    1.013334
1    14.228917  3.972904e+06    1.683687
2    15.929588  2.350741e+06    2.813271
3    16.596866  2.277155e+06    2.774887
4    17.540535  2.453705e+06    1.190322
5    14.675349  3.518300e+06    1.693567
6    15.231850  2.794933e+06    1.242961
7    17.464224  2.455910e+06    1.157568
8    16.889858  2.315323e+06    2.140472
9    16.789886  2.418906e+06    1.970405
10   16.862590  2.408533e+06    1.456068
11   16.626252  2.280497e+06    2.273390
12   15.035113  2.405980e+06    2.554789
13   16.086162  2.363632e+06    2.675050
14   14.978454  2.792946e+06    1.704016
15   15.495462  2.435764e+06    2.216868
16   14.949365  2.378213e+06    2.640432
17   15.698612  2.444640e+06    1.699472
18   17.469172  2.454862e+06    1.099878
19   15.027633  2.405705e+06    2.555526


AttributeError: 'DataFrame' object has no attribute 'append'

## EMA Workbench

In [ ]:
def model_wrapper(**kwargs):
    input = [kwargs['v' + str(i)] for i in range(len(kwargs))]
    Hydropower, Environment, Irrigation = tuple(ZambeziProblem.evaluate(np.array(input)))
    return Hydropower, Environment, Irrigation


In [ ]:
from ema_workbench import (RealParameter, ScalarOutcome, Constant,
                           Model)

model = Model('zambeziproblem', function=model_wrapper)

model.levers = [RealParameter('v' + str(i), -1, 1) for i in range(ZambeziProblem.Nvar)]

#specify outcomes
model.outcomes = [ScalarOutcome('Hydropower', ScalarOutcome.MINIMIZE),
                  ScalarOutcome('Environment', ScalarOutcome.MINIMIZE),
                  ScalarOutcome('Irrigation', ScalarOutcome.MINIMIZE)]

In [ ]:
print(len(model.outcomes))

3


KeyError: 10

In [ ]:
from ema_workbench import MultiprocessingEvaluator, SequentialEvaluator, ema_logging
import dill

ema_logging.log_to_stderr(ema_logging.INFO)

with MultiprocessingEvaluator(model) as evaluator:
    results = evaluator.optimize(nfe=100,  searchover='levers', epsilons=[0.1, 0.1, 0.1])

In [ ]:
from ema_workbench import MultiprocessingEvaluator, SequentialEvaluator, IpyparallelEvaluator, ema_logging
from ema_workbench.em_framework.ema_ipyparallel import (start_logwatcher, set_engine_logger,
                                  initialize_engines, cleanup, _run_experiment)
import dill
from ipyparallel import Client


rc = Client()

ema_logging.log_to_stderr(ema_logging.INFO)

with IpyparallelEvaluator(model, client=rc) as evaluator:
    results = evaluator.optimize(nfe=100, searchover='levers',
    epsilons=[0.1,]*len(model.outcomes))

Waiting for connection file: ~\.ipython\profile_default\security\ipcontroller-client.json


OSError: Connection file '~\\.ipython\\profile_default\\security\\ipcontroller-client.json' not found.
You have attempted to connect to an IPython Cluster but no Controller could be found.
Please double-check your configuration and ensure that a cluster is running.